## Possible formula for destruction:

Group continents into 6 categories, Americas (N & S), Europe, Africa, Asia, Australasia & Antarctica.

They align with STR, DEX, CON, INT, WIS, CHA respectively.

Flat formula - Size(S1, M2, L3) X d20 roll X continent mulitplier = Damage.

In [1]:
import pandas as pd
import random

file = pd.read_csv("./dnd_monsters.csv")
cc_file = pd.read_csv("./countries_continents.csv")

size_dict = {"Tiny":0.5,
    "Small":1,
    "Medium":2,
    "Large":3,
    "Huge":4,
    "Gargantuan":5
    }

# file.iloc[random.randint(0, 200)]['hp']

# sort out missing values for dex con int etc
file = file.fillna(random.randint(0, 5))

# make a new column for size multiplier
file['size_multiplier'] = file['size'].map(size_dict)

# D20 dice roll
D20 = random.randint(0, 20)

In [2]:
pop_file = pd.read_csv("./population_by_country_2020.csv")

pop_file = pop_file[
    ['Country (or dependency)', 'Population (2020)']
]
merged_df = pd.merge(cc_file, pop_file, left_on='Country', right_on='Country (or dependency)')

merged_df['Population'] = merged_df['Population (2020)']
merged_df.pop('Population (2020)')

merged_df

,Continent,Country,Country (or dependency),Population
0,Africa,Algeria,Algeria,43984569
1,Africa,Angola,Angola,33032075
2,Africa,Benin,Benin,12175480
3,Africa,Botswana,Botswana,2359585
4,Africa,Burundi,Burundi,11948454
...,...,...,...,...
178,South America,Paraguay,Paraguay,7147553
179,South America,Peru,Peru,33050211
180,South America,Suriname,Suriname,587541
181,South America,Uruguay,Uruguay,3475842


In [12]:
import certifi
import ssl
import geopy.geocoders
from geopy.geocoders import Nominatim
import random
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

def get_continent(Nominatim=Nominatim):
    geolocator = Nominatim(user_agent="my_geocoder")

    land = False
    while land != True:
        lat = random.randint(-90, 90)
        long = random.randint(-180, 180)

        location = geolocator.reverse((lat, long), language='en')
        try:
            country = location.raw['address']['country']
            land = True
            # print(f"Country: {country}")

            continent_row = cc_file.loc[cc_file['Country']==country]

            continent = continent_row['Continent'].to_list()[0]
            # print(f"Continent: {continent}")
        except:
            print("Monster drowned in the briny depths of the ocean.")
            land = False

    return country, continent

country, continent = get_continent()

In [8]:



continent

'North America'

Group continents into 6 categories, Americas (N & S), Europe, Africa, Asia, Australasia & Antarctica.

They align with STR, DEX, CON, INT, WIS, CHA respectively.

Flat formula - Size(S1, M2, L3) X d20 roll X continent mulitplier = Damage.

In [9]:
mod = ["str", "dex", "con", "int", "wis", "cha"]
continent_dict = {cc_file['Continent'].unique()[i]: mod[i] for i in range(len(mod))}

continent_dict


{'Africa': 'str',
 'Asia': 'dex',
 'Europe': 'con',
 'North America': 'int',
 'Oceania': 'wis',
 'South America': 'cha'}

In [10]:
def create_monster_damage(file=file, continent_dict=continent_dict, continent=continent):
    rand_monster = file.sample()
    monster_name = rand_monster['name'].iloc[0]

    damage = 1000 * D20 * rand_monster['size_multiplier'] * rand_monster[continent_dict[continent]]

    damage = int(damage.iloc[0])

    return monster_name, damage

create_monster_damage()

('barghest', 429000)

## Possible repurcussions from damage.

Damage could equal population killed. Bit bleak? Try and evacuate people in time? Then we could work towards live data. 

Damage has three stages, 6 seconds, 6 minutes, 60 minutes.

Might be difficult to get population of cities and locales.

Could do population of countries, and can move people to a bordering country. Would be interesting to keep a running population table.

In [ ]:
# population = merged_df['Population (2020)'].loc[merged_df['Country']==country]
# population = int(population.iloc[0])
# percent_loss = round(damage / population * 100, 3)


# print(f"Country: {country}, \nPopulation: {population}, \nMonster: {monster_name.upper()}, \nDamage: {damage} \nUpdated Population: {population - damage}, \nPercent Population Lost: {percent_loss}")


# Create fake sample stream

In [13]:
import time
import json
import datetime
from threading import Thread


for i in range(8):
    # time delay
    time.sleep(1)

    # unpack monster info
    monster_name, damage = create_monster_damage()

    # unpack country info
    country, continent = get_continent()
    # country, continent = "Canada", "North America"

    # Update the main DF in order to keep track of running casualties in countries.
    index_loc = merged_df.loc[merged_df['Country']==country].index[0]

    # population info
    population = int(merged_df['Population'].at[index_loc])


    # display results
    # print(f"Country: {country}, \nPopulation: {population}, \nMonster: {monster_name.upper()}, \nDamage: {damage} \nUpdated Population: {population - damage}, \nPercent Population Lost: {percent_loss}\n\n---\n")

    # \|/ BELOW IS THE CORRECT WAY TO AMEND VALUES IN A DATAFRAME \|/

    # need to catch minus population numbers 
    if population - damage < 0:
        updated_population = 0
        merged_df.at[index_loc, 'Population'] = 0
    else:
        updated_population = population - damage
        merged_df.at[index_loc, 'Population'] -= damage

    # print(f"Pop: {population}")
    # print(f"UpPop: {updated_population}")
    
    # calculate percentage population loss !!! CATCH percentages greater than 100
    percent_loss = round(damage / population * 100, 3)
    if percent_loss > 100:
        percent_loss = 100

    # structure data for stream
    data = {
        "country": country,
        "population": population,
        "monster_name": monster_name.upper(),
        "damage": damage,
        "updated_population": int(updated_population),
        "percent_loss": percent_loss,
        "ts": str(datetime.datetime.now())
    }
    # jsonify
    json_string = json.dumps(data)
    print(json_string)
    # print(f"--- {updated_population} ---")

    time.sleep(1)
 

{"country": "China", "population": 1440253825, "monster_name": "QUADRONE", "damage": 22000, "updated_population": 1440231825, "percent_loss": 0.002, "ts": "2023-09-26 09:40:28.080567"}
Monster drowned in the briny depths of the ocean.
Monster drowned in the briny depths of the ocean.
Monster drowned in the briny depths of the ocean.
Monster drowned in the briny depths of the ocean.
Monster drowned in the briny depths of the ocean.
Monster drowned in the briny depths of the ocean.
Monster drowned in the briny depths of the ocean.
Monster drowned in the briny depths of the ocean.
{"country": "Mexico", "population": 129166028, "monster_name": "PENTADRONE", "damage": 33000, "updated_population": 129133028, "percent_loss": 0.026, "ts": "2023-09-26 09:40:34.224111"}
Monster drowned in the briny depths of the ocean.
Monster drowned in the briny depths of the ocean.
{"country": "Greenland", "population": 56787, "monster_name": "BLUE-ABISHAI", "damage": 22000, "updated_population": 34787, "perc

## I need a function to take the random monster and country info and times the damage by ten and write it to the stream

This is because I cannot have the monsters being created in a game loop, and still be times 10, then times 100 with the damage stages whilst the loop is running as all the variables will be redefined.

This will possibly be executed with Threading and targets, but for now I will deprioritize.

In [ ]:
# # damage, updated_population = damage_stage(2, damage, updated_population)
#     # damage_stage(3, damage, updated_population)

#     t1 = Thread(target = damage_stage(2))
#     # t2 = Thread(target = secondFunction)

#     t1.start()
#     # t2.start()

In [ ]:
 # STAGE 2

    # # damage increases after 6 minutes
    # ten_damage = damage * 10
    # damage += ten_damage

    # # recalculate population with 0 catch
    # if updated_population - ten_damage < 0:
    #     updated_population = 0
    # else:
    #     updated_population = updated_population - ten_damage

    # # recalculate percent loss 
    # percent_loss = round((population - updated_population) / population * 100, 3)

    # data = {
    #     "country": country,
    #     "population": population,
    #     "monster_name": monster_name.upper(),
    #     "damage": damage,
    #     "updated_population": (updated_population),
    #     "percent_loss": percent_loss,
    #     "ts": str(datetime.datetime.now())
    # }
    # # jsonify
    # json_string = json.dumps(data)
    # print(json_string)

## Depriotized `damage_stage()`

In [ ]:
# def damage_stage(stage, damage=damage, updated_population=updated_population):
#     if stage == 2:
#         time.sleep(5) #!test
#         # damage increases after 6 minutes
#         print(damage)
#         ten_damage = damage * 10
#         damage += ten_damage
#         print(damage) 
#         # recalculate population with 0 catch
#         if updated_population - ten_damage < 0:
#             updated_population = 0
#         else:
#             updated_population = updated_population - ten_damage
#     elif stage == 3:
#         time.sleep(5) #!test
#         # damage increases after 6 minutes
#         print(damage)
#         hundred_damage = damage * 10
#         damage += hundred_damage
#         print(damage)
#         if updated_population - hundred_damage < 0:
#             updated_population = 0
#         else:
#             updated_population = updated_population - hundred_damage


   

#     # recalculate percent loss 
#     percent_loss = round((population - updated_population) / population * 100, 3)

#     data = {
#         "country": country,
#         "population": population,
#         "monster_name": monster_name.upper(),
#         "damage": damage,
#         "updated_population": (updated_population),
#         "percent_loss": percent_loss,
#         "ts": str(datetime.datetime.now())
#     }
#     # jsonify
#     json_string = json.dumps(data)
#     print(json_string)

#     return damage, updated_population

## I'm running into an issue with multiple while loops...

Without getting into the mess that is threading how do I execute a loop to make a continuous stream of monsters and have EACH of those monsters do *10 damage after 6 minutes, then each of those monsters do *100 damage after an hour.

- Perhaps this amount of damage is actually too much at this stage 
- Should I re-prioritize to make sure I can post out the monster damage in a faux stream environment and...
- Keep track of the damage that is being done to countries.